In [40]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [41]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [42]:
%run -i ../../zkstats/core.py

In [43]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [44]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: arbitraryyy, just to make sure after filtered, it's not empty
dummy_data = np.random.uniform(1, 100, len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

# where(element > 30)
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
dummy_theory_output = torch.mean(gt30_dummy_data_tensor)

In [45]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        num_fil_X = torch.sum((X>30).double())
        fil_X = torch.where(X>30, X, 0.0)
        return (torch.abs(torch.sum(fil_X)-num_fil_X*(self.w))<=torch.abs(0.01*num_fil_X*self.w), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [46]:
# prover calculates settings, send to verifier
gt30_data_tensor = data_tensor[data_tensor > 30].reshape(1,-1,1)
print("new tensor: ", gt30_data_tensor.size())
# gt30_data_tensor = torch.reshape(gt30_data_tensor_raw, (1, int(gt30_data_tensor_raw.size()[0]), 1))
theory_output = torch.mean(gt30_data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        
    def forward(self,X):
        # where part
        num_fil_X = torch.sum((X>30).double())
        fil_X = torch.where(X>30, X, 0.0)
        return (torch.abs(torch.sum(fil_X)-num_fil_X*(self.w))<=torch.abs(0.01*num_fil_X*self.w), self.w)
    

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [2], "resources", settings_path)

new tensor:  torch.Size([1, 272, 1])
Theory_output:  tensor(52.3676)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-160256,1254],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":3318,"total_const_size":306,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [47]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 29.613948822021484 seconds
Theory output:  tensor(52.3676)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 52.25
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12572659313263335624, 14443766455855958404, 432930639589567449, 1881177029071802301], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18278699600166517679, 8643075271396760825, 7891176514265388517, 2236426588013702123]]], 'proof': '10cf5e22c2c80d762d56a2d23f9bf4ae2cb3ae50dc25cc9db468beb558350b04255eab97ed1419fcc8c811c4c1395558c6485bf6bf2bd81ab4b833aa5f210c6b2761aaf6b55ae614c08dfef12350ae528520cad8961524395ce8f3a53e9d42e413424920e5708129faa83115d2a16f4ba9ce062be499208c21012a8fbb25ac2d0ae8a720f8a6bf0e58a3bfef42e256eb1df5797913f005f010c3f9b04b8443ef082eb353fd01bf11f1cb92d91c6e3547e24325b43a35e6a66875f063f73f85b722c5a147646dc67b6603e95ef7f72c9f6a8a46ce6e77c0fe353a68a6ac037d1b127c5d6331ec06c4adefccdfd9f3afb517ced0ee555e0f38385ba1ad8b1fda2516250a4bb7b25e980e6e86919540dff18df22426dc2f50bfab883152a1707a0707828f68cf86230b31fa274da59520d30408697eb959d9f6d93171f4fe28b82a0cb4fadc380a020eb1fc29c473b635313b956bd52c71be6c406110d6bc397c1

In [49]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[12572659313263335624, 14443766455855958404, 432930639589567449, 1881177029071802301], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18278699600166517679, 8643075271396760825, 7891176514265388517, 2236426588013702123]]]
proof boolean:  1.0
proof result 1 : 52.25
verified
